In [6]:
import pandas as pd

direccion = "./../data/raw/prueba.csv"
df = pd.read_csv(direccion)

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.csv(direccion, header="true", inferSchema="true")
# te mostrará los primerros 20 renglones
df.show()

+----+-------+-----+----------+---------+-------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-------------+---------+-------------+--------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----

In [8]:
#%pyspark
#Leer data
#df = spark.read.format("csv").option("header", "true").load("s3a://aws-logs-032274842050-us-east-1/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_11.csv")


#Pasar a minusculas los nombres de columnas
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

#Seleccionar columnas no vacias 

base = df.select(df.year,df.quarter, df.month, df.dayofmonth, df.dayofweek, df.flightdate, df.reporting_airline, df.dot_id_reporting_airline, df.iata_code_reporting_airline, df.tail_number, df.flight_number_reporting_airline, df.originairportid, df.originairportseqid, df.origincitymarketid, df.origin, df.origincityname, df.originstate, df.originstatefips, df.originstatename, df.originwac, df.destairportid, df.destairportseqid, df.destcitymarketid, df.dest, df.destcityname, df.deststate, df.deststatefips, df.deststatename, df.destwac, df.crsdeptime, df.deptime, df.depdelay, df.depdelayminutes, df.depdel15, df.departuredelaygroups, df.deptimeblk, df.taxiout, df.wheelsoff, df.wheelson, df.taxiin, df.crsarrtime, df.arrtime, df.arrdelay, df.arrdelayminutes, df.arrdel15, df.arrivaldelaygroups, df.arrtimeblk, df.cancelled, df.diverted, df.crselapsedtime, df.actualelapsedtime, df.airtime, df.flights, df.distance, df.distancegroup, df.divairportlandings )

#agregar columna con clasificación de tiempo en horas de atraso del vuelo 0-1.5, 1.5-3.5,3.5-, cancelled

from pyspark.sql import functions as f
base = base.withColumn('rangoatrasohoras', f.when(f.col('cancelled') == 1, "cancelled").when(f.col('depdelayminutes') < 90, "0-1.5").when((f.col('depdelayminutes') > 90) & (f.col('depdelayminutes')<210), "1.5-3.5").otherwise("3.5-"))

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, lower, regexp_replace, split

#Función limpieza
def clean_text(c):
    c = lower(c)
    c = regexp_replace(c, " ", "_")
    c = f.split(c, '\,')[0]
    return c


 # Aplicación de la función limpieza
base = base.withColumn("origincityname", clean_text(col("origincityname")))
base = base.withColumn("destcityname", clean_text(col("destcityname")))


In [11]:
base.head()

Row(year=2019, quarter=4, month=11, dayofmonth=16, dayofweek=6, flightdate=datetime.datetime(2019, 11, 16, 0, 0), reporting_airline='OH', dot_id_reporting_airline=20397, iata_code_reporting_airline='OH', tail_number='N517AE', flight_number_reporting_airline=5213, originairportid=13795, originairportseqid=1379502, origincitymarketid=33795, origin='OAJ', origincityname='jacksonville/camp_lejeune', originstate='NC', originstatefips=37, originstatename='North Carolina', originwac=36, destairportid=11057, destairportseqid=1105703, destcitymarketid=31057, dest='CLT', destcityname='charlotte', deststate='NC', deststatefips=37, deststatename='North Carolina', destwac=36, crsdeptime=1036, deptime=1032, depdelay=-4.0, depdelayminutes=0.0, depdel15=0.0, departuredelaygroups=-1, deptimeblk='1000-1059', taxiout=9.0, wheelsoff=1041, wheelson=1119, taxiin=8.0, crsarrtime=1201, arrtime=1127, arrdelay=-34.0, arrdelayminutes=0.0, arrdel15=0.0, arrivaldelaygroups=-2, arrtimeblk='1200-1259', cancelled=0.0

In [13]:
base['rangoatrasohoras']

Column<b'rangoatrasohoras'>